# Jakarta Data Demo

Loosly based on https://github.com/gavinking/data-demo

In [ ]:
//DEPS com.h2database:h2:2.3.232
//DEPS jakarta.data:jakarta.data-api:1.0.1
//DEPS org.hibernate.orm:hibernate-core:7.0.2.Final
//DEPS org.hibernate.orm:hibernate-jpamodelgen:7.0.2.Final



In [ ]:
import jakarta.persistence.Entity;
import jakarta.persistence.Id;
import jakarta.persistence.GeneratedValue;
import jakarta.persistence.GenerationType;

@Entity
class Book {
    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    private String title;
    private String author;
}


In [ ]:

import jakarta.data.repository.*;

@Repository
public interface Library {

    @Insert
    void addBook(Book book);

    @Query("where title like :title")
    List<Book> findByTitle(String title);
    
}

In [ ]:
import org.hibernate.jpa.HibernatePersistenceConfiguration;

var config = new HibernatePersistenceConfiguration("Jakarta Data Example")
.managedClasses(Book.class)
.property("hibernate.dialect", "org.hibernate.dialect.H2Dialect")
.property("hibernate.connection.url", "jdbc:h2:mem:test")
.property("hibernate.connection.username", "sa")
.property("hibernate.connection.password", "");

try (var sessionFactory = config.createEntityManagerFactory()) {
    var schemaManager = sessionFactory.getSchemaManager();
    schemaManager.drop(true);
    schemaManager.create(true);

    sessionFactory.inStatelessSession(session -> {
        // repository is usually injected via CDI,
        // but here we just instantiate it
        final Library library = new Library_(session);

        // load a Book
        var book = library.book("9781932394153");
        out.println(book);
    });
}


